# TP : Régression Logistique (from scratch) — Méthode de la Descente de Gradient

Dans ce notebook, nous implémentons la régression logistique entièrement depuis les formules mathématiques, sans utiliser `sklearn` pour l'entraînement.

Nous utiliserons le dataset *Pima Indians Diabetes Database* (Kaggle), où la variable cible est binaire (diagnostic diabète = 1 / non diabétique = 0), et nous choisirons **une seule variable explicative** pour respecter les consignes du TP.

L’objectif est de :
- dériver la log-vraisemblance,
- calculer son gradient,
- coder une descente de gradient (maximisation),
- entraîner le modèle et comparer avec `sklearn`.


Référence : https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database.

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LogisticRegression

# Charger le dataset (téléchargez pima-indians-diabetes.csv et placez-le dans data/)
df = pd.read_csv('diabetes.csv')
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## Modèle statistique et log-vraisemblance

Nous considérons le modèle logistique :

$$
p(x_i) = \sigma(\beta_0 + \beta_1 x_i)
= \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_i)}}.
$$

Pour des données $(x_i, y_i)$ où $y_i \in \{0,1\}$, la log-vraisemblance est :

$$
\ell(\beta_0,\beta_1)
= \sum_{i=1}^n \left[
y_i \log(p_i) + (1-y_i)\log(1 - p_i)
\right].
$$

### Gradient de la log-vraisemblance

En dérivant :

$$
\frac{\partial \ell}{\partial \beta_0}
= \sum_{i=1}^n (y_i - p_i),
$$

$$
\frac{\partial \ell}{\partial \beta_1}
= \sum_{i=1}^n x_i (y_i - p_i).
$$

Notons :

$$
\nabla \ell(\beta)
=
\begin{pmatrix}
\sum (y_i - p_i) \\
\sum x_i (y_i - p_i)
\end{pmatrix}.
$$

### Mise à jour de la descente de gradient (maximisation)

Comme on maximise $\ell$ :

$$
\beta^{(t+1)} = \beta^{(t)} + \eta \, \nabla \ell(\beta^{(t)}).
$$

où $\eta > 0$ est le pas d’apprentissage.



## Préparation des données

Nous utilisons le dataset Pima Diabetes.

### Pourquoi standardiser la variable explicative ?

Avant d’entraîner un modèle logistique par optimisation numérique, il est essentiel de **standardiser** la variable explicative :

$$
X_{\text{std}} = \frac{X - \mu_X}{\sigma_X}.
$$

Cette transformation ramène les données à une moyenne 0 et un écart-type 1.

En optimisation, cela apporte plusieurs avantages :
- la descente de gradient est plus stable et plus rapide ;
- l’algorithme évite les oscillations (‘zigzag’) lorsque l’échelle des données est grande ;
- le choix du learning rate devient plus simple et moins sensible.


Pour respecter la consigne du TP, nous ne retenons qu'une seule variable explicative, par exemple :
- `Glucose`
- (autres options possibles : BMI, Age, BloodPressure…)

Nous séparons ensuite les données en deux échantillons : apprentissage et test.


In [4]:
feature = 'Glucose'
X = df[[feature]].values.astype(float)
y = df['Outcome'].values.astype(int)

# Standardiser X
scaler = StandardScaler()
Xs = scaler.fit_transform(X).reshape(-1)  # vecteur shape (n,)

# split
X_train, X_test, y_train, y_test = train_test_split(Xs, y, test_size=0.5, random_state=0)
n = len(X_train)


## Implémentation du modèle logistique et de son gradient

Nous définissons :

- la fonction logistique (sigmoid),
- la log-vraisemblance,
- le gradient,
- puis la descente de gradient.

L’objectif est de maximiser $\ell(\beta)$.


In [10]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def log_vraisemblance(beta, X, y):
    # beta: array([b0, b1])
    z = beta[0] + beta[1]*X
    p = sigmoid(z)
    eps = 1e-12
    return np.sum(y*np.log(p+eps) + (1-y)*np.log(1-p+eps))

def gradient(beta, X, y):
    z = beta[0] + beta[1]*X
    p = sigmoid(z)
    g0 = np.sum(y - p)
    g1 = np.sum(X * (y - p))
    return np.array([g0, g1])


In [11]:
def gradient_ascent(X, y, lr=0.01, max_iter=2000, tol=1e-6, verbose=False):
    beta = np.zeros(2)
    history = {'ll':[], 'beta':[]}
    for k in range(max_iter):
        g = gradient(beta, X, y)
        beta = beta + lr * g
        ll = log_vraisemblance(beta, X, y)
        history['ll'].append(ll)
        history['beta'].append(beta.copy())
        if np.linalg.norm(lr*g) < tol:
            if verbose: print(f"Converged at iter {k}")
            break
        # optional: reduce lr if ll decreases (simple safeguard)
        if k>0 and history['ll'][-1] < history['ll'][-2]:
            lr *= 0.5
    return beta, history

beta_gd, hist_gd = gradient_ascent(X_train, y_train, lr=0.01, max_iter=5000, verbose=True)
beta_gd


Converged at iter 22


array([-0.75108379,  1.14686835])

## Analyse des performances

Nous évaluons :
- l'accuracy,
- l'AUC,
- et comparons nos coefficients estimés avec ceux de `sklearn.linear_model.LogisticRegression`.

Cela permet de valider l'exactitude de notre implémentation from scratch.


In [7]:
# prédictions
def predict_prob(beta, X):
    return sigmoid(beta[0] + beta[1]*X)

def predict(beta, X, thr=0.5):
    return (predict_prob(beta, X) >= thr).astype(int)

y_pred_train = predict(beta_gd, X_train)
y_pred_test  = predict(beta_gd, X_test)

print("Train accuracy:", accuracy_score(y_train, y_pred_train))
print("Test  accuracy:", accuracy_score(y_test, y_pred_test))
print("Test AUC:", roc_auc_score(y_test, predict_prob(beta_gd, X_test)))


Train accuracy: 0.75
Test  accuracy: 0.7421875
Test AUC: 0.7981021633527441


In [9]:
# comparaison avec sklearn (fit intercept = True)
X_train_2d = X_train.reshape(-1,1)
X_test_2d  = X_test.reshape(-1,1)
clf = LogisticRegression(solver='lbfgs').fit(X_train_2d, y_train)
beta_sklearn = np.array([clf.intercept_[0], clf.coef_[0,0]])
print("Sklearn beta:", beta_sklearn)
print("Notre beta   :", beta_gd)


Sklearn beta: [-0.74658524  1.12416839]
Notre beta   : [-0.75108379  1.14686835]
